# PUMS Household Income vs. AMI (2021) in Phoenix  PUMAS

- https://www.census.gov/data/developers/data-sets/

For households by income and household size to compare to HUD AMI in same year
-  https://api.census.gov/data/2021/acs/acs1/pums/variables.html

2021 AMI by HH Size (from City of Phoenix)
- https://www.phoenix.gov/humanservicessite/Documents/2021%20AMI%20Limits%204.2.21.pdf

In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
import get_pums as get
import pums as calc

In [3]:
#Search parameters
y1 = '2021'

sample = 'acs1'

phx_pumas = ['0400113','0400114','0400115','0400116','0400117',\
             '0400118','0400119','0400120','0400121','0400122','0400123',\
             '0400125','0400128','0400112','0400129']

#north_pumas = ['0400112','0400129']

data_cols = 'SERIALNO,ST,PUMA,HINCP,NP,WGTP,ADJINC,ADJHSG,SMOCP'

ADJHSG - adjustment factor for housing dollar amounts (6 decimal places)
1000000 = 1.000000

FHINCP - income flag - 1: yes | https://api.census.gov/data/2021/acs/acs1/pums/variables/FHINCP.json

HINCP - HHI in past 12 months (not -60000:n/a, 0:no income, -59999 loss of 59k+)
-1 to -59998 loss, 1+
https://api.census.gov/data/2021/acs/acs1/pums/variables/HINCP.json

SMOCP - selected owner costs (monthly) (not '00000': none / '-1')
https://api.census.gov/data/2021/acs/acs1/pums/variables/SMOCP.json

In [4]:
#AMI bands in 2021 by household size
AMI_30pct = {'1':16600,'2':19000,'3':21960,'4':26500,'5':31040,'6':35580,\
            '7':40120,'8':44660}
AMI_50pct = {'1':27650,'2':31600,'3':35500,'4':39500,'5':42700,'6':45850,\
            '7':49000,'8':52150}
AMI_80pct = {'1':44250,'2':50600,'3':56900,'4':63200,'5':68300,'6':73350,\
            '7':78400,'8':83450}
AMI_100pct = {'1':55300,'2':63200,'3':71100,'4':79000,'5':85400,'6':91700,\
            '7':98000,'8':104300}

In [5]:
#Housing costs affordable to different AMI bands based on household size
aff_1p = {'30pct':461,'50pct':767,'80pct':1217,'100pct':1535}
aff_2p = {'30pct':527,'50pct':877,'80pct':1392,'100pct':1754}
aff_3p = {'30pct':609,'50pct':985,'80pct':1565,'100pct':1973}
aff_4p = {'30pct':735,'50pct':1096,'80pct':1738,'100pct':2192}
aff_5p = {'30pct':861,'50pct':1185,'80pct':1878,'100pct':2370}
aff_6p = {'30pct':987,'50pct':1272,'80pct':2017,'100pct':2545}
aff_7p = {'30pct':1113,'50pct':1360,'80pct':2156,'100pct':2720}
aff_8p = {'30pct':1239,'50pct':1447,'80pct':2295,'100pct':2894}

unit_afford = {'1':[0,461,767,1217,1535,1000000000],\
              '2':[0,527,877,1392,1535,1000000000],\
              '3':[0,609,985,1565,1973,1000000000],\
              '4':[0,735,1096,1738,2192,1000000000],\
              '5':[0,861,1185,1878,2370,1000000000],\
              '6':[0,987,1272,2017,2545,1000000000],\
              '7':[0,1113,1360,2156,2720,1000000000],\
              '8':[0,1239,1447,2295,2894,1000000000]}

inc_lbls = ['u30_ami','30_50_ami','50_80_ami','80_100_ami','o100_ami']

In [6]:
# create a list of replicate weights
repwt = 'WGTP'
repwts = [repwt+str(i) for i in range(1, 81)]

## Get PUMA data

In [7]:
df = get.get_puma(sample,y1,data_cols)

In [8]:
df['GEO_ID'] = df['ST']+df['PUMA']
df = df[df.GEO_ID.isin(phx_pumas)]
df  = df.drop(['SERIALNO','ST','PUMA'],axis=1)
df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
for col in df.columns[1:]: df[col] = df[col].astype(float)

In [9]:
df['HHSz'] = pd.cut(df['NP'],bins=[0,1,2,3,4,5,6,7,14],\
                   labels=['1','2','3','4','5','6','7','8'])
df['HHSz'] = df['HHSz'].astype(str)
df['HINCP'] = df.ADJINC * df.HINCP

In [10]:
dff = df[~(df.HHSz.isna())&(df.HINCP!=-60000)&(df.HINCP!=0)&(df.SMOCP!=0)&(df.SMOCP!=-1)].copy()

In [11]:
dff['hou_cost'] = dff.SMOCP
dff['aff_cost'] = np.where(dff.HINCP>=1,(dff.HINCP*0.333)/12,0)

In [12]:
#assign AMI range based on household size and ami dictionaries
dff['AMI_range'] = np.where((dff['HINCP']<=dff['HHSz'].map(AMI_30pct)),'u30_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_30pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_50pct)),'30_50_ami',\
                   np.where((dff['HINCP']>dff['HHSz'].map(AMI_50pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_80pct)),\
                   '50_80_ami',np.where((dff['HINCP']>dff['HHSz'].map(AMI_80pct))&(dff['HINCP']<=dff['HHSz'].map(AMI_100pct)),\
                   '80_100_ami','o100_ami'))))

In [13]:
dff['cost_burdened'] = np.where(dff.hou_cost>dff.aff_cost,'burdened','not burdened')

In [14]:
#who is the unit affordable to based on the rent
dff['unit_aff'] = np.where(dff.HHSz=='1',pd.cut(dff['hou_cost'],bins=unit_afford['1'],labels=inc_lbls),\
                  np.where(dff.HHSz=='2',pd.cut(dff['hou_cost'],bins=unit_afford['2'],labels=inc_lbls),\
                  np.where(dff.HHSz=='3',pd.cut(dff['hou_cost'],bins=unit_afford['3'],labels=inc_lbls),\
                  np.where(dff.HHSz=='4',pd.cut(dff['hou_cost'],bins=unit_afford['4'],labels=inc_lbls),\
                  np.where(dff.HHSz=='5',pd.cut(dff['hou_cost'],bins=unit_afford['5'],labels=inc_lbls),\
                  np.where(dff.HHSz=='6',pd.cut(dff['hou_cost'],bins=unit_afford['6'],labels=inc_lbls),\
                  np.where(dff.HHSz=='7',pd.cut(dff['hou_cost'],bins=unit_afford['7'],labels=inc_lbls),\
                  np.where(dff.HHSz=='8',pd.cut(dff['hou_cost'],bins=unit_afford['8'],labels=inc_lbls),''))))))))

In [15]:
dff.head(3)

,GEO_ID,HINCP,NP,WGTP,ADJINC,ADJHSG,SMOCP,WGTP1,WGTP2,WGTP3,...,WGTP77,WGTP78,WGTP79,WGTP80,HHSz,hou_cost,aff_cost,AMI_range,cost_burdened,unit_aff
3602,0400112,76214.6720,2.0,55.0,1.029928,1000000.0,3176.0,56.0,96.0,91.0,...,93.0,51.0,56.0,94.0,2,3176.0,2114.957148,o100_ami,burdened,o100_ami
3607,0400116,138010.3520,6.0,53.0,1.029928,1000000.0,1628.0,15.0,16.0,62.0,...,18.0,90.0,58.0,54.0,6,1628.0,3829.787268,o100_ami,not burdened,50_80_ami
3618,0400122,32339.7392,2.0,109.0,1.029928,1000000.0,717.0,96.0,152.0,114.0,...,216.0,235.0,169.0,186.0,2,717.0,897.427763,50_80_ami,not burdened,30_50_ami


### table by PUMA for renters by AMI range - cost burdened vs. not cost burdened

In [16]:
def make_est(df):
    df['hh_SE'] = df.apply(lambda x: (calc.get_se(x['WGTP'],x[repwts])),axis=1)
    df['hh_MOE'] = df.apply(lambda x: (calc.get_moe(x['hh_SE'])),axis=1)
    df['hh_CV'] = df.apply(lambda x: (calc.get_cv(x['WGTP'],x['hh_SE'])),axis=1)
    df.rename(columns={'WGTP':'hh'},inplace=True)
    return df

In [17]:
drop_cols = ['HINCP','NP','ADJINC','ADJHSG','SMOCP','hou_cost','aff_cost','HHSz']

In [18]:
table = dff.copy().drop(columns=drop_cols)

In [19]:
table = table.groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()

In [20]:
table_2 = table.copy().groupby(['GEO_ID','AMI_range','unit_aff','cost_burdened']).sum().reset_index()
table_2 = make_est(table_2)
table_2 = table_2.drop(columns=repwts)

In [21]:
own_ami = table.copy().drop(columns=['unit_aff','cost_burdened']).groupby(['GEO_ID','AMI_range']).sum().reset_index()
own_ami = make_est(own_ami)
own_ami = own_ami.drop(columns=repwts)

In [22]:
own_aff = table.copy().drop(columns=['AMI_range','cost_burdened']).groupby(['GEO_ID','unit_aff']).sum().reset_index()
own_aff = make_est(own_aff)
own_aff = own_aff.drop(columns=repwts)

In [23]:
cost_burd = table.copy().drop(columns=['unit_aff','AMI_range']).groupby(['GEO_ID','cost_burdened']).sum().reset_index()
cost_burd = make_est(cost_burd)
cost_burd = cost_burd.drop(columns=repwts)

In [24]:
table_3 = table.copy().drop(columns='cost_burdened').groupby(['GEO_ID','AMI_range','unit_aff']).sum().reset_index()
table_3 = make_est(table_3)
table_3 = table_3.drop(columns=repwts)

In [25]:
table_2 = pd.pivot_table(table_2,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff','cost_burdened'],aggfunc=np.sum).reset_index()

NameError: name 'rent_ami' is not defined

In [26]:
own_ami = pd.pivot_table(own_ami,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range'],aggfunc=np.sum).reset_index()

own_aff = pd.pivot_table(own_aff,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['unit_aff'],aggfunc=np.sum).reset_index()

cost_burd = pd.pivot_table(cost_burd,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['cost_burdened'],aggfunc=np.sum).reset_index()

In [27]:
table_3 = pd.pivot_table(table_3,values=['hh','hh_MOE','hh_CV'],index='GEO_ID',\
                          columns=['AMI_range','unit_aff'],aggfunc=np.sum).reset_index()

In [29]:
with pd.ExcelWriter(f'output/pums_grossrent_gap.xlsx') as writer:
    table_2.to_excel(writer,sheet_name='full_owner')
    own_ami.to_excel(writer,sheet_name='hh_ami_lvl_owner')
    own_aff.to_excel(writer,sheet_name='hh_afford_byAMI_owner')
    cost_burd.to_excel(writer,sheet_name='cost_burdened_byAMI_owner')
    table_3.to_excel(writer,sheet_name='demand_supply_amilvl')